In [1]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head()

total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [3]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [4]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [5]:
##Split independent and dependent features
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df['total_bill']


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,
                                                     random_state=42)

In [7]:
X_train.head()

tip     sex smoker  day    time  size
115  3.50  Female     No  Sun  Dinner     2
181  5.65    Male    Yes  Sun  Dinner     2
225  2.50  Female    Yes  Fri   Lunch     2
68   2.01    Male     No  Sat  Dinner     2
104  4.08  Female     No  Sat  Dinner     2

## Feature Encoding (Label Encoding and One Hot Encoding)

In [8]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

In [9]:
X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])
X_train.head()

tip  sex  smoker  day  time  size
115  3.50    0       0  Sun     0     2
181  5.65    1       1  Sun     0     2
225  2.50    0       1  Fri     1     2
68   2.01    1       0  Sat     0     2
104  4.08    0       0  Sat     0     2

In [10]:
X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])
X_test.head()

tip  sex  smoker   day  time  size
24   3.18    1       0   Sat     0     2
6    2.00    1       0   Sun     0     2
153  2.00    1       0   Sun     0     4
211  5.16    1       1   Sat     0     4
198  2.00    0       1  Thur     1     2

In [11]:
# OHE with Column Transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('onehot', OneHotEncoder(drop='first'), [3])]
                       ,remainder='passthrough')

In [12]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
X_train = ct.fit_transform(X_train)

In [13]:
X_test = ct.transform(X_test)
X_test

array([[1.  , 0.  , 0.  , 3.18, 1.  , 0.  , 0.  , 2.  ],
       [0.  , 1.  , 0.  , 2.  , 1.  , 0.  , 0.  , 2.  ],
       [0.  , 1.  , 0.  , 2.  , 1.  , 0.  , 0.  , 4.  ],
       [1.  , 0.  , 0.  , 5.16, 1.  , 1.  , 0.  , 4.  ],
       [0.  , 0.  , 1.  , 2.  , 0.  , 1.  , 1.  , 2.  ],
       [0.  , 1.  , 0.  , 2.  , 1.  , 1.  , 0.  , 2.  ],
       [0.  , 0.  , 1.  , 2.56, 1.  , 1.  , 1.  , 2.  ],
       [0.  , 0.  , 1.  , 2.52, 0.  , 0.  , 1.  , 2.  ],
       [0.  , 1.  , 0.  , 3.23, 1.  , 0.  , 0.  , 2.  ],
       [0.  , 0.  , 0.  , 3.  , 0.  , 1.  , 0.  , 2.  ],
       [0.  , 1.  , 0.  , 3.  , 1.  , 0.  , 0.  , 2.  ],
       [1.  , 0.  , 0.  , 1.47, 1.  , 0.  , 0.  , 2.  ],
       [0.  , 0.  , 1.  , 1.5 , 0.  , 0.  , 1.  , 2.  ],
       [0.  , 1.  , 0.  , 2.  , 1.  , 1.  , 0.  , 2.  ],
       [0.  , 0.  , 1.  , 1.83, 0.  , 0.  , 1.  , 1.  ],
       [0.  , 0.  , 1.  , 1.36, 0.  , 0.  , 1.  , 3.  ],
       [0.  , 0.  , 1.  , 4.  , 1.  , 1.  , 1.  , 3.  ],
       [0.  , 1.  , 0.  , 3.92,

## Training using SVR

In [14]:
from sklearn.svm import SVR
svr = SVR()

In [15]:
svr.fit(X_train, y_train)

SVR()

In [16]:
y_pred = svr.predict(X_test)

In [17]:
from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.49798620106004743
4.463296539661224


## Hyperparameter tuning

In [18]:
from sklearn.model_selection import GridSearchCV

params = {'C':[0.1,1,10,100,1000],
          'gamma':[1,0.1,0.01,0.001,0.0001],
          'kernel':['rbf','sigmoid','linear']}

grid_cv = GridSearchCV(SVR(), param_grid=params, cv=5, refit=True, verbose=3)

grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.114 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.008 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.003 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.015 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.123 total time=   0.0s
[CV 1/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.162 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.038 total time=   0.0s
[CV 3/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.032 total time=   0.0s
[CV 4/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.036 total time=   0.0s
[CV 5/5] END ...C=0.1, gamma=1, kernel=sigmoid;, score=-0.165 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.502 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'sigmoid', 'linear']},
             verbose=3)

In [19]:
grid_cv.best_params_

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

In [20]:
y_pred = grid_cv.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.5630727637616448
4.26048397387227
